### 손글씨 이미지 데이터 MINST
- 많은 이미지 데이터 학습
- 학습후 새로운 데이터 입력시 판별
- 지도 학습 
- 데이터는 학습용 6만개, 테스트 전용 1만개
- http://yann.lecun.com/exdb/mnist/
 > train-images-idx3-ubyte.gz:  training set images (9912422 bytes) 
 > train-labels-idx1-ubyte.gz:  training set labels (28881 bytes) 
 > t10k-images-idx3-ubyte.gz:   test set images (1648877 bytes) 
 > t10k-labels-idx1-ubyte.gz:   test set labels (4542 bytes)

|절차|단계|목표/설명|
|:---|---:|:---:|
|1|연구목표|- 손글씨 이미지(숫자:0~9) 학습시켜서, 이후에 새로운 손글씨 이미지를 판별해내는 머신러닝 모듈 구현이 포인트<br>- 이미지 데이터를 디코딩하여, 이미지 하나당 구성된 총 픽셀수 및 <br>사이즈를 계산하여 이미지를 추출한다 => 디코딩과정 => 6만개의 데이터를 생성<br>- 이미지와 같은 바이너리 데이터를 백터화하는 과정|
|2|데이터획득/수집|- http://yann.lecun.com/exdb/mnist/ 접속<br>- 데이터수집 LEVER3 Web Scraping을 활용하여 압축 데이터를 받는 URL을 획득<br>- 압축데이터를 직접 요청하여 받고 저장하면서 압축 풀고 등 일련의 과정 진행|
|3|데이터준비/통찰/전처리|- MNIST database 구조 이해<br>- 디코딩처리가 가능<br>- 엔디언(Endiann) 내용 추가로 이해|
|4|데이터탐색/통찰/시각화||
|5|데이터모델링 or 모델구축||
|6|시스템통합 or (솔루션, 서비스, 레포트)||

#### 2-1 데이터획득/수집

- 2-1. 수집할 데이터 URL 획득

In [1]:
import urllib.request as req
from bs4 import BeautifulSoup

In [6]:
baseUrl = 'http://yann.lecun.com/exdb/mnist/'
soup = BeautifulSoup( req.urlopen( baseUrl ), 'html5lib')

In [7]:
soup

<html><head>
   <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
   <title>MNIST handwritten digit database, Yann LeCun, Corinna Cortes and Chris Burges</title>
<script type="text/javascript">
var gaJsHost = (("https:" == document.location.protocol) ? "https://ssl." : "http://www.");
document.write(unescape("%3Cscript src='" + gaJsHost + "google-analytics.com/ga.js' type='text/javascript'%3E%3C/script%3E"));
</script><script type="text/javascript">
var pageTracker = _gat._getTracker("UA-6178702-1");
pageTracker._trackPageview();
</script></head>




<body alink="#111111" background="wm5b.gif" bgcolor="#FFFFFF" link="#FF0000" vlink="#820594">

<center>
<h1>
<font color="#FF0000"><font size="+4">THE MNIST DATABASE</font></font></h1></center>

<center><font color="#FF0000"><font size="+4">of handwritten digits</font></font></center>

<center><font color="#FF0000"><font size="+4"></font></font></center>

<center><font color="#3366FF"><font size="+1"><a href="http://yann

In [19]:
soup.findAll('tt')[:4]

[<tt><a href="train-images-idx3-ubyte.gz">train-images-idx3-ubyte.gz</a>: 
 training set images (9912422 bytes)</tt>,
 <tt><a href="train-labels-idx1-ubyte.gz">train-labels-idx1-ubyte.gz</a>: 
 training set labels (28881 bytes)</tt>,
 <tt><a href="t10k-images-idx3-ubyte.gz">t10k-images-idx3-ubyte.gz</a>:  
 test set images (1648877 bytes)</tt>,
 <tt><a href="t10k-labels-idx1-ubyte.gz">t10k-labels-idx1-ubyte.gz</a>:  
 test set labels (4542 bytes)</tt>]

In [18]:
for tt in soup.findAll('tt')[:4]:
    # href 값이나, 링크의 문자열이나 동일하다
    print( tt.a.string )
    # 링크는 안나오는 이유는 해당 주소에서 바로 데이터를 바로 내려받는 곳이기 때문이다(주소 바로 밑에 포함되어있는 형태)

train-images-idx3-ubyte.gz
train-labels-idx1-ubyte.gz
t10k-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz


In [20]:
# gz 파일 4개 url 획득 => 리스트에 담는다
files = [tt.a.string for tt in soup.findAll('tt')[:4]]
files

['train-images-idx3-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz']

- 2-2. 압축데이터를 다운로드

In [21]:
import os, os.path, gzip

In [23]:
# 파일 저장 위치 선정
savePath = './data/mnist'
# 해당 디렉토리가 없으면 생성해라
# 디렉토리 관련해서 os를 import를 시킨것이다
if not os.path.exists(savePath):  # 물리적으로 존재하는가? 존재안한다(존재하지않는다): false(not)
    #os.mkdir( savePath ) # 디렉토리 만드는 코드, 여기서 오류나서 makedirs로 코드 변경실시함(체킹)
    os.makedirs(savePath)

In [27]:
# 저장
from tqdm import tqdm_notebook # tqdm:진행률
for file in tqdm_notebook(files):
    print( '원본데이터주소', baseUrl + file) # 원본주소랑 붙인다: baseUrl의 주소의 맨 끝에 /가 있어서 file과 합칠 수 있다
    
    local_path = '%s/%s' % (savePath , file)
    print( '저장될 데이터의 경로', '%s/%s' % (savePath , file)) # 저장될 주소 : savePath에 붙일때 /가 없어서 %를 사용하여 붙이고 사용함
    
    # 이미 파일이 존재하면 새로 받지 않겠다
    if not os.path.exists(local_path):
        # 웹상에 존재하는 리소스를 로컬에 직접 저장(위치 지정)
        req.urlretrieve( baseUrl + file, local_path ) # 밑으로 떨어뜨리는 것(저장) : local_path

원본데이터주소 http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
저장될 데이터의 경로 ./data/mnist/train-images-idx3-ubyte.gz
원본데이터주소 http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
저장될 데이터의 경로 ./data/mnist/train-labels-idx1-ubyte.gz
원본데이터주소 http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
저장될 데이터의 경로 ./data/mnist/t10k-images-idx3-ubyte.gz
원본데이터주소 http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
저장될 데이터의 경로 ./data/mnist/t10k-labels-idx1-ubyte.gz



- 2-3. 압축 데이터 압축 해제: gzip

In [29]:
# 파일에서 10k:1만개 
# 이름규칙 => train-images-idx3-ubyte.gz => 압축해제를 하면: train-images-idx3-ubyte 로서 .gz가 빠진다
for file in tqdm_notebook(files):
    # 원본 파일
    ori_file = '%s/%s' % (savePath , file) 
    # 압축해제 파일(.gz 만 빠지면 된다)
    raw_file = '%s/%s' % (savePath , file[:-3] )  # 문자열 벗겨냄: .replace
    print( ori_file, raw_file ) # raw_file : 압축해제 파일이 될 것이다.
    # <압축파일을 오픈 -> 읽어서 -> 기록>
    # 1. 압축파일 오픈
    with gzip.open( ori_file, 'rb' ) as fg: # rb : 바이너리 파일을 읽어오기
        # 내용을 읽고
        # 만약 대용량이면, 분할해서 읽어야 한다
        tmp = fg.read()
        with open( raw_file, 'wb' ) as f: # wb : 바이너리 파일을 쓰기
            f.write( tmp )

./data/mnist/train-images-idx3-ubyte.gz ./data/mnist/train-images-idx3-ubyte
./data/mnist/train-labels-idx1-ubyte.gz ./data/mnist/train-labels-idx1-ubyte
./data/mnist/t10k-images-idx3-ubyte.gz ./data/mnist/t10k-images-idx3-ubyte
./data/mnist/t10k-labels-idx1-ubyte.gz ./data/mnist/t10k-labels-idx1-ubyte



### 3. 데이터준비/통찰/전처리

- 디비구조 이해상 나온 내용 :
- 엔디언/에디언 (Endiann) : 컴퓨터 메모리와 같은 1차원의 공간에 여러개의 연속된 데이터를 배열하는 방법 -> 오더링: ordering -> 바이트 오더: byte order
- 값: 0x12345678
- 빅에디언:

|메모리|...|0x100|0x101|0x102|0x103|...|
|:---|---:|:---:|
|값|...|0x12|0x34|0x56|0x78|...|

- 리틀에디언:

|메모리|...|0x100|0x101|0x102|0x103|...|
|:---|---:|:---:|
|값|...|0x78|0x56|0x34|0x12|...|

### label의 구조
- 구조
<br> > magic number : 4 byte
<br> > label 개수   : 4 byte
<br> > label        : 1 byte ....
 


### image의 구조
- 구조
<br> > magic number : 4 byte
<br> > image 개수   : 4 byte
<br> > 이미지 가로  : 4 byte
<br> > 이미지 세로  : 4 byte
<br> > 이미지       : 28 * 28 byte ....

In [ ]:
# 레이블데이터, 원데이터를 읽어서 => 풀어서 데이터를 저장 

In [ ]:
# t10k-images-idx3-ubyte
# t10k-labels-idx1-ubyte
# train-images-idx3-ubyte
# train-labels-idx1-ubyte
# 훈련용 데이터, 테스트용 데이터 각각 처리 -> 같은 구조로 2세트로 가니깐 -> 함수로 처리해야 한다(반복되지 않는다)
# 실데이터는 어디에 저장할 것인가? -> csv 저장해보겠다. 그리고, 저장도 하겠다(*.pgm)

In [32]:
import struct

In [57]:
# 파일 여는 곳
def decode_minist_rawData( dataType='train', maxCnt=0 ): #기본값 0을 준다, maxCnt 인자를 집어넣어서 idx카운트가 maxCnt를 초과하는 순간 멈추게 할 것이다.
    # 레이블 파일 읽기 모드 오픈
    label_f = open('%s/%s-labels-idx1-ubyte' % ( savePath, dataType ), 'rb')
    # 이미지 데이터 파일 읽기 모드 오픈 
    image_f = open('%s/%s-images-idx3-ubyte' % ( savePath, dataType ), 'rb')
    # csv 쓰기 모드 오픈(train.csv)
    csv_f   = open('%s/%s.csv' % ( savePath, dataType ), 'w', encoding='utf-8') # csv는 텍스트: 그냥 'r'
    #-----------------------------------------------------------------------------------------------------
    
    # 디코딩을 위한 해더 정보 추출
    # 매직코드나 사이즈, 이미지의 가로, 세로 정보등을 획득하기 위해서는
    # 4바이트씩 읽어서 빅에디안(high endian) 처리를 해야한다 -> struct 모듈해결 : struct사용하기 위해서 import 시킨다
    # label
    # >II (대문자 I) => 읽은 총 데이터를 2번(로) 나눠서 각각 변수에 담아라
    # > => 빅에디안
    magic, label_count = struct.unpack( '>II', label_f.read(8) ) # 두개의 내용이 리턴된다는 뜻(튜플로), label_f.read(8): 다 읽는게 아닌 8바이트 읽는다
    print( magic, label_count )
    # image, 파일포인터(파일 커서)의 위치는 16바이트를 가리키고 있을것이다. 
    magic, image_count, row, col = struct.unpack('>IIII', image_f.read(16))
    print( magic, image_count, row, col )
    # 이미지 한개당 총 픽셀수
    pixels = row * col # 픽셀수 = 가로 곱하기 세로
    
    
    # 데이터를 추출(디코딩)해서 csv에 저장
    # 이미지 갯수만큼 돌아야함
    for idx in range( label_count ): # 6만번 계산을 할것이다.
        # 데이터를 원하는 만큼 세팅했다
        if idx >= maxCnt:break  # idx가 특정횟수에 도달하면 멈출것이다., break: 수행문이 하나면 옆으로 이어쓸 수 있다
        #-----------------------------------------------------------------------------------------------------------
        
        # 정답추출 -> 1 byte만 읽으면 된다. -> unsigned byte -> 언사인드바이트 기호: 'B'
        label_tmp = struct.unpack('B', label_f.read(1) ) 
        print( label_tmp )
        # 값이 결측치에 해당될 경우 -> 이미지 데이터로 같이 날린다
        if len(label_tmp) != 1:
            continue
        # 라벨값, 정답 획득
        label = label_tmp[0] # 레이블은 정수값을 가지고 있다
        #---------------------------------------------------------------------------------------------------------
        
        # 이미지 추출 => 얼만큼 읽어야 하는가?
        # 이미지 데이터 원본은 에디안하고는 관계 없다. 여러바이트를 쓰는
        # 정수가 아닌 데이터 자체이므로 무관하다
        binarryData = image_f.read( pixels )
        print( type(binarryData)) # binarryData의 타입확인하는 구문
        #break  
        # 픽셀값 하나 하나를 문자열로 만들어서 리스트에 담아라
        #strData = list(map( 함수, 데이터 ))
        strData = list(map( lambda x: str(x), binarryData ))
        # csv에 기록은 : label, 이미지데이터 줄바꿈
        csv_f.write( str(label)+',' )
        # 이미지데이터 줄바꿈까지
        csv_f.write( ','.join( strData )+'\n' ) # ,로 조인하겠다는 뜻
        #print(strData)
        #break
        #---------------------이까지 csv파일로 떨어뜨린다
        
        # 해당 데이터를 별도로 저장하여 이미지로 파일로 보관
        # 대략 10개정도만 저장할 것이다 (너무 많으면 정신없으니 10개로 줄임)
        if idx < 10:
            # pqm파일로 저장할 것이다 -> 파일명도 정한다
            fName  = '{0}/{1}-{2}-{3}.pgm'.format( savePath, dataType, idx, label ) # {}:경로/{}:파일명
            # pqm 헤더
            header = 'P2 28 28 255\n' # P2 : 매직넘어 일것이다, 가로, 세로, 255:색상의 수(색은 0부터 255까지이다)
            s      = header + ' '.join( strData )   # 공백으로 조인할 것이다, strData:배열
            
            # 파일기록
            with open( fName, 'w', encoding='utf-8') as f:
                f.write( s ) # 해당 이미지도 떨어뜨릴수 있을 것이다 
            
        
        
    # 파일 닫기
    label_f.close()
    image_f.close()
    csv_f.close()    

In [58]:
# ,.join 예제
','.join(['0','0','0','0','0'])

'0,0,0,0,0'

In [59]:
# 함수를 호출해서 사용한다
# 
decode_minist_rawData( dataType='train', maxCnt=750) # 2049: 매직넘버, 6만개: label카운트
decode_minist_rawData( dataType='t10k', maxCnt=250) # 750바퀴 250바퀴 도는데 10개의 정도로 맞출것이다(위의 소스코드에서) 

2049 60000
2051 60000 28 28
(5,)
<class 'bytes'>
(0,)
<class 'bytes'>
(4,)
<class 'bytes'>
(1,)
<class 'bytes'>
(9,)
<class 'bytes'>
(2,)
<class 'bytes'>
(1,)
<class 'bytes'>
(3,)
<class 'bytes'>
(1,)
<class 'bytes'>
(4,)
<class 'bytes'>
(3,)
<class 'bytes'>
(5,)
<class 'bytes'>
(3,)
<class 'bytes'>
(6,)
<class 'bytes'>
(1,)
<class 'bytes'>
(7,)
<class 'bytes'>
(2,)
<class 'bytes'>
(8,)
<class 'bytes'>
(6,)
<class 'bytes'>
(9,)
<class 'bytes'>
(4,)
<class 'bytes'>
(0,)
<class 'bytes'>
(9,)
<class 'bytes'>
(1,)
<class 'bytes'>
(1,)
<class 'bytes'>
(2,)
<class 'bytes'>
(4,)
<class 'bytes'>
(3,)
<class 'bytes'>
(2,)
<class 'bytes'>
(7,)
<class 'bytes'>
(3,)
<class 'bytes'>
(8,)
<class 'bytes'>
(6,)
<class 'bytes'>
(9,)
<class 'bytes'>
(0,)
<class 'bytes'>
(5,)
<class 'bytes'>
(6,)
<class 'bytes'>
(0,)
<class 'bytes'>
(7,)
<class 'bytes'>
(6,)
<class 'bytes'>
(1,)
<class 'bytes'>
(8,)
<class 'bytes'>
(7,)
<class 'bytes'>
(9,)
<class 'bytes'>
(3,)
<class 'bytes'>
(9,)
<class 'bytes'>
(8,)
<

<class 'bytes'>
(9,)
<class 'bytes'>
(2,)
<class 'bytes'>
(2,)
<class 'bytes'>
(3,)
<class 'bytes'>
(8,)
<class 'bytes'>
(1,)
<class 'bytes'>
(6,)
<class 'bytes'>
(5,)
<class 'bytes'>
(1,)
<class 'bytes'>
(1,)
<class 'bytes'>
(0,)
<class 'bytes'>
(2,)
<class 'bytes'>
(6,)
<class 'bytes'>
(4,)
<class 'bytes'>
(5,)
<class 'bytes'>
(8,)
<class 'bytes'>
(3,)
<class 'bytes'>
(1,)
<class 'bytes'>
(5,)
<class 'bytes'>
(1,)
<class 'bytes'>
(9,)
<class 'bytes'>
(2,)
<class 'bytes'>
(7,)
<class 'bytes'>
(4,)
<class 'bytes'>
(4,)
<class 'bytes'>
(4,)
<class 'bytes'>
(8,)
<class 'bytes'>
(1,)
<class 'bytes'>
(5,)
<class 'bytes'>
(8,)
<class 'bytes'>
(9,)
<class 'bytes'>
(5,)
<class 'bytes'>
(6,)
<class 'bytes'>
(7,)
<class 'bytes'>
(9,)
<class 'bytes'>
(9,)
<class 'bytes'>
(3,)
<class 'bytes'>
(7,)
<class 'bytes'>
(0,)
<class 'bytes'>
(9,)
<class 'bytes'>
(0,)
<class 'bytes'>
(6,)
<class 'bytes'>
(6,)
<class 'bytes'>
(2,)
<class 'bytes'>
(3,)
<class 'bytes'>
(9,)
<class 'bytes'>
(0,)
<class 'bytes

(4,)
<class 'bytes'>
(0,)
<class 'bytes'>
(1,)
<class 'bytes'>
(3,)
<class 'bytes'>
(1,)
<class 'bytes'>
(3,)
<class 'bytes'>
(4,)
<class 'bytes'>
(7,)
<class 'bytes'>
(2,)
<class 'bytes'>
(7,)
<class 'bytes'>
(1,)
<class 'bytes'>
(2,)
<class 'bytes'>
(1,)
<class 'bytes'>
(1,)
<class 'bytes'>
(7,)
<class 'bytes'>
(4,)
<class 'bytes'>
(2,)
<class 'bytes'>
(3,)
<class 'bytes'>
(5,)
<class 'bytes'>
(1,)
<class 'bytes'>
(2,)
<class 'bytes'>
(4,)
<class 'bytes'>
(4,)
<class 'bytes'>
(6,)
<class 'bytes'>
(3,)
<class 'bytes'>
(5,)
<class 'bytes'>
(5,)
<class 'bytes'>
(6,)
<class 'bytes'>
(0,)
<class 'bytes'>
(4,)
<class 'bytes'>
(1,)
<class 'bytes'>
(9,)
<class 'bytes'>
(5,)
<class 'bytes'>
(7,)
<class 'bytes'>
(8,)
<class 'bytes'>
(9,)
<class 'bytes'>
(3,)
<class 'bytes'>
(7,)
<class 'bytes'>
(4,)
<class 'bytes'>
(6,)
<class 'bytes'>
(4,)
<class 'bytes'>
(3,)
<class 'bytes'>
(0,)
<class 'bytes'>
(7,)
<class 'bytes'>
(0,)
<class 'bytes'>
(2,)
<class 'bytes'>
(9,)
<class 'bytes'>
(1,)
<class '

#### 4. 데이터탐색/통찰/시각화

- 데이터는 현재  csv에 저장되어 있다. 이를 로드해서 데이터를 전처리 하여 학습에 적합한 구조로 정리

In [25]:
# 저장된 csv 파일 경로
savePath + '/' + 'train' + '.csv', savePath + '/' + 't10k' + '.csv'

('./data/mnist/train.csv', './data/mnist/t10k.csv')

In [26]:
# 1. csv를 오픈
# 2. csv를 읽는다 -> 데이터 한줄(row)을 읽어서 리스트에 맴버로 담는다
#    ex) train.csv를 리스트에 750개의 맴버가 생성, t10k는 250개 생성
#    [ [],[],[],[], ]
# 3. 리스트의 맴버(리스트)는 0번맴버가 레이블, 1번~ 마지막까지가 픽셀인데, 
#    픽셀은 현재 0 ~255까지 값으로 구성, 학습 성능을 높이기 위해 
#    픽셀값을 0 ~ 1(255)사이로 정규화 하여 배치. 1, 28*28개
# { 'labels':labels, 'images':images }

In [27]:
def load_csv( dataType='train' ):
    # 1. csv를 오픈
    fName = '{}/{}.csv'.format( savePath, dataType )
    print( fName )
    labels = []
    images = []
    with open( fName, 'r' ) as f:
        # 2. csv를 읽는다 
        #print( len( f.readline() ) )
        for line in f: # for문을 이용하여 한줄씩 리드 하는 표현
            #print( type(line), line )
            # 데이터를 들여다 보니, 785개의 데이터는 모두 ,로 구분되어있다->분해
            tmp = line.split(',')
            #print( len(tmp), type( int(tmp[0]) ) )
            # labels에 답(0~9) 데이터를 정수값으로 세팅 -> 1차 리스트
            labels.append( int(tmp[0]) )
            #break            
            # images에 [ 28*28개의 픽셀값 ] 맴버로 추가 -> 2차 리스트
            #images.append( tmp[1:] )
            images.append( list( map( lambda x: int(x)/256, tmp[1:] ) ) )
            
            # => 단, 데이터는 0~1까지 정규화 진행된 값
            # 255로 정규화, 256으로 정규화하는 것인가?
            # 데이터 자체를 들여다 보면 => 픽셀 데이터는 앞뒤로 영향을 받는 
            # 연속적인 성향을가진 값인자, 독립적인 값인지 검토
            # 수치값 -> 연속변수(255:최대값기준), 분류변수(256:총개수기준)
            
            # python => list( map( 함수, 데이터덩어리 ) )
            # pandas => 데이터덩어리 = 데이터덩어리.apply( 함수 )
    
    
    return { 'labels':labels, 'images':images  }

In [28]:
# 나름 최적화 
def load_csv( dataType='train' ):    
    fName = '{}/{}.csv'.format( savePath, dataType )    
    labels = []
    images = []
    with open( fName, 'r' ) as f:        
        for line in f:
            tmp = line.split(',')            
            # tmp.pop(0) : 0번 맴버를 제거하고, 리턴
            labels.append( int(tmp.pop(0)) )        
            # len(tmp) => 784개로 1개 줄어들었다
            images.append( list( map( lambda x: int(x)/256, tmp ) ) )            
    return { 'labels':labels, 'images':images  }

In [29]:
train = load_csv(  )
test  = load_csv( dataType='t10k' )

#### 5. 데이터모델링 or 모델구축

In [30]:
# 알고리즘 선택 => 파이프라인 구축 
from sklearn import svm, model_selection, metrics

In [31]:
# 알고리즘 생성 => 파이프라인 구축, 하이퍼파라미터 튜닝
clf = svm.SVC() # 기본값 들어감

In [ ]:
# 데이터분류 (이미 되어 있음)

In [32]:
# 학습
clf.fit( train['images'], train['labels'])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [33]:
# 예측 (테스트 데이터 사용)
predict = clf.predict( test['images'] )

In [34]:
# 결과 확인, 결과 리포트 => 성능 평가
ac_score = metrics.accuracy_score( test['labels'], predict ) # 답을 넣는다
ac_score

0.788

In [35]:
cl_report = metrics.classification_report( test['labels'], predict )
print( cl_report )

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        19
          1       0.79      1.00      0.88        34
          2       0.70      0.67      0.68        24
          3       0.86      0.52      0.65        23
          4       0.79      0.79      0.79        33
          5       0.68      0.84      0.75        25
          6       1.00      0.64      0.78        22
          7       0.81      0.76      0.79        29
          8       0.92      0.79      0.85        14
          9       0.65      0.81      0.72        27

avg / total       0.80      0.79      0.79       250



In [36]:
# 풀세트 구성 후 => 반복작업 : 본 칸에 반복작업 구성
decode_minist_rawData( dataType='train',  maxCnt=60000 )
decode_minist_rawData( dataType='t10k',   maxCnt=10000 )


clf = svm.SVC()

train = load_csv(  )
test  = load_csv( dataType='t10k' )

clf.fit( train['images'], train['labels'])

predict = clf.predict( test['images'] )

ac_score = metrics.accuracy_score( test['labels'], predict ) # 답을 넣는다
print(ac_score)

cl_report = metrics.classification_report( test['labels'], predict )
print( cl_report )





2049 60000
2051 60000 28 28
2049 10000
2051 10000 28 28
0.9443
             precision    recall  f1-score   support

          0       0.96      0.99      0.97       980
          1       0.97      0.99      0.98      1135
          2       0.94      0.93      0.93      1032
          3       0.93      0.94      0.93      1010
          4       0.93      0.96      0.94       982
          5       0.93      0.91      0.92       892
          6       0.95      0.97      0.96       958
          7       0.96      0.93      0.94      1028
          8       0.94      0.92      0.93       974
          9       0.94      0.92      0.93      1009

avg / total       0.94      0.94      0.94     10000

